# Quickstart

This notebook is just to make sure that everything works. Firstly, create a file called `.env` in this folder, and add the following content, obviously with your values. There are two ways to authenticate: using a token or using the Azure CLI (AAD authentication). The Azure CLI is the recommended way.

```
OPENAI_API_KEY=xxxxxx
OPENAI_API_BASE=https://xxxxxxx.openai.azure.com/
```

When authenticating using Azure CLI + Azure AD, open a terminal and run the following command: `az login --use-device-code`. This will open a browser window where you can authenticate. After authentication, use Option 1 code to run and you are good to go.

*(You could as well run the following command to get the token: `az account get-access-token --resource https://management.core.windows.net/ --query accessToken -o tsv`. Copy the token and add it to the `.env` file and then use the token to authenticate.)*

> **🔔 Please note that in all following notebooks you will need to set the authentication method accordingly!**

In [ ]:
import os
import tiktoken
import openai
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from openai.embeddings_utils import cosine_similarity
from tenacity import retry, wait_random_exponential, stop_after_attempt

# Load environment variables
load_dotenv()

# Option 1 - Use Azure AD authentication with az cli (use az login in terminal)
default_credential = DefaultAzureCredential()
token = default_credential.get_token("https://cognitiveservices.azure.com/.default")
openai.api_type = "azure_ad"
openai.api_base = os.environ.get("OPENAI_API_BASE")
openai.api_key = token.token
openai.api_version = "2022-12-01"

# Option 2 - Using Access Key
# openai.api_type = "azure"
# openai.api_base = os.environ.get("OPENAI_API_BASE")
# openai.api_key = os.environ.get("OPENAI_API_KEY")
# openai.api_version = "2022-12-01"

# Define embedding model and encoding
EMBEDDING_MODEL = 'text-embedding-ada-002'
COMPLETION_MODEL = 'text-davinci-003'
encoding = tiktoken.get_encoding('cl100k_base')

Test if completitions work:

In [ ]:
response = openai.Completion.create(engine="text-davinci-003",
                                    prompt="Knock knock.",
                                    temperature=0)
print(response.choices[0].text)

Do it in a streaming fashion:

In [ ]:
import sys
for resp in openai.Completion.create(engine='text-davinci-003', prompt='Give me 10 taglines for an ice cream shop', max_tokens=512, stream=True):
    sys.stdout.write(resp.choices[0].text)
    sys.stdout.flush()

Test if embeddings work:

In [ ]:
e = openai.Embedding.create(input="Hello World!", engine=EMBEDDING_MODEL)["data"][0]["embedding"]
print(e)

Test if tokenizer works:

In [ ]:
tokens = encoding.encode("Hello world!")
print(tokens)
print(len(tokens))

Also let's check if LangChain works:

In [ ]:
import os
import openai
from dotenv import load_dotenv
from langchain.llms import AzureOpenAI
from langchain.embeddings import OpenAIEmbeddings

# Load environment variables
load_dotenv()

# Configure Azure OpenAI Service API
openai.api_type = "azure"
openai.api_version = "2022-12-01"
openai.api_base = os.getenv('OPENAI_API_BASE')
openai.api_key = os.getenv("OPENAI_API_KEY")

# Create a completion
llm = AzureOpenAI(deployment_name="text-davinci-003")
joke = llm("Tell me a dad joke")
print(joke)

# Create embeddings
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", chunk_size=1)
text = "This is a test document."

# Embed a single document
query_result = embeddings.embed_query(text)
print(query_result)

# Embed multiple documents at once
doc_result = embeddings.embed_documents([text, text])